In [1]:
import wandb

import os, warnings
from fastai.vision.all import *
from sklearn.model_selection import StratifiedGroupKFold
from pathlib import Path

WANDB_PROJECT = "mlops-course-001"

RAW_DATA_AT = 'bdd_simple_1k'
PROCESSED_DATA_AT = 'bdd_simple_1k_split'

warnings.filterwarnings('ignore')

/home/m_bobrin/anaconda3/envs/RL/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/m_bobrin/anaconda3/envs/RL/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


In [2]:
!pip install wandb --upgrade

In [3]:
run = wandb.init(project=WANDB_PROJECT, job_type='data_split')

wandb: Currently logged in as: simmax21. Use `wandb login --relogin` to force relogin


In [4]:
raw_data_at = run.use_artifact('bdd_simple_1k_split:latest')
path = Path(raw_data_at.download(root="/home/m_bobrin/trash_data"))

wandb: Downloading large artifact bdd_simple_1k_split:latest, 846.59MB. 4007 files... 
wandb:   4007 of 4007 files downloaded.  
Done. 0:0:6.0


In [5]:
path.ls()

(#5) [Path('/home/m_bobrin/trash_data/media'),Path('/home/m_bobrin/trash_data/eda_table.table.json'),Path('/home/m_bobrin/trash_data/LICENSE.txt'),Path('/home/m_bobrin/trash_data/labels'),Path('/home/m_bobrin/trash_data/images')]

In [6]:
orig_table = raw_data_at.get("eda_table")

wandb: Downloading large artifact bdd_simple_1k_split:latest, 846.59MB. 4007 files... 
wandb:   4007 of 4007 files downloaded.  
Done. 0:0:6.1


In [10]:
fnames = orig_table.get_column('File_Name')
groups = [s.split('-')[0] for s in fnames]

y = orig_table.get_column('bicycle')

In [11]:
df = pd.DataFrame()
df['File_Name'] = fnames
df['fold'] = -1

In [13]:
cv = StratifiedGroupKFold(n_splits=10)
for i, (train_idx, test_idx) in enumerate(cv.split(fnames, y, groups)):
    df.loc[test_idx, ['fold']] = i

In [14]:
df['Stage'] = 'train'
df.loc[df.fold == 0, ['Stage']] = 'test'
df.loc[df.fold == 1, ['Stage']] = 'valid'

del df['fold']
df.Stage.value_counts()

train    800
valid    100
test     100
Name: Stage, dtype: int64

In [19]:
df.to_csv('data_split.csv', index=False)
processed_data = wandb.Artifact('bdd_simple_1k_processed', type='split_data')
processed_data.add_file('data_split.csv')
processed_data.add_dir(path)

wandb: Adding directory to artifact (/home/m_bobrin/trash_data)... Done. 0.9s


In [20]:
data_split_table = wandb.Table(dataframe=df[['File_Name', 'Stage']])
join_table = wandb.JoinedTable(orig_table, data_split_table, 'File_Name')
processed_data.add(join_table, "data_split_table")

<ManifestEntry digest: Y9Js/64jMfdPNKKQNsmFfg==>

In [21]:
run.log_artifact(processed_data)
run.finish()